# LeetCode exercise
This is a solution for an specific requirement (taxi company trips) available in [LeetCode](https://leetcode.com/problems/trips-and-users/description/).
This excercise works as a pandas training! I hope you enjoy it as much as I did! 

**Note:** Please refer to the embedded link to get the full description of the problem.

## Algorithm
**Understanding the problem.**

We are asked to calculate the cancellation rate of requests with unbanned users between "2013-10-01" and "2013-10-03". We are also supposed to only take dates with at least one trip.


**Create input dfs for testing**

Using the given schema and input example, we'll create the input tables.

In [134]:
import pandas as pd

# Trips DataFrame
trips_df = pd.DataFrame({
    'id': [1,2,3,4,5,6,7,8,9,10],
    'client_id': [1,2,3,4,1,2,3,3,2,4],
    'driver_id': [10,11,12,13,10,11,12,12,12,13],
    'city_id': [1,1,6,6,1,6,6,12,12,12],
    'status': [
        'completed',
        'cancelled_by_driver',
        'completed',
        'cancelled_by_client',
        'completed',
        'completed',
        'completed',
        'completed',
        'completed',
        'cancelled_by_driver'
    ],
    'request_at': [
        '2013-10-01',
        '2013-10-01',
        '2013-10-01',
        '2013-10-01',
        '2013-10-02',
        '2013-10-02',
        '2013-10-02',
        '2013-10-03',
        '2013-10-03',
        '2013-10-03'
    ]
})

# Users DataFrame
users_df = pd.DataFrame({
    'users_id': [1,2,3,4,10,11,12,13],
    'banned': ['No','Yes','No','No','No','No','No','No'],
    'role': ['client','client','client','client','driver','driver','driver','driver']
})

In [135]:
trips_df

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,3,12,12,completed,2013-10-03
8,9,2,12,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [136]:
users_df

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


**Write the trips_and_users function**

This function is in charge of calculating the cancellation rate:

As you'll see below, this function involves filtering the data to fullfill the specified conditions.


In [ ]:
def trips_and_users(trips: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    """This function calculates the cancellation rate of requests with unbanned users between "2013-10-01" and "2013-10-03".

    Args:
        trips (pd.DataFrame): df that contains trip data.
        users (pd.DataFrame): df that contains user data.

    Returns:
        pd.DataFrame: df with the cancellation rate of requests with unbanned users between "2013-10-01" and "2013-10-03".
    """
    default_df = pd.DataFrame(columns=['Day', 'Cancellation Rate'])
    # Convert 'request_at' to datetime
    trips['request_at'] = pd.to_datetime(trips['request_at'])
    # Filter trips for the specified date range
    trips_filtered = trips[(trips['request_at'] >= '2013-10-01') & (trips['request_at'] <= '2013-10-03')]
    # Merge the tables to get a banned or not banned column
    trips_filtered = trips_filtered.merge(users[['users_id', 'banned']], left_on='client_id', right_on='users_id', how='left')
    trips_filtered = trips_filtered.rename(columns={'banned': 'client_banned'})
    trips_filtered = trips_filtered.drop('users_id', axis=1)
    # Merge for driver info
    trips_filtered = trips_filtered.merge(users[['users_id', 'banned']], left_on='driver_id', right_on='users_id', how='left')
    trips_filtered = trips_filtered.rename(columns={'banned': 'driver_banned'})
    trips_filtered = trips_filtered.drop('users_id', axis=1)
    # Filter out trips where either client or driver is banned
    trips_filtered = trips_filtered[(trips_filtered['client_banned'] == 'No') & (trips_filtered['driver_banned'] == 'No')]

    if trips_filtered.empty or trips.empty or users.empty:
        return default_df
    else:
        # Group by date and calculate cancellation rate
        return trips_filtered.groupby('request_at').apply(
                        lambda group : pd.Series({'Day' : group['request_at'].iloc[0].strftime('%Y-%m-%d'),
                        'Cancellation Rate' : round(group[group['status'].str.contains('cancelled')].shape[0]/group.shape[0], 2)})).reset_index(drop=True)
        

    


In [138]:
trips_and_users(trips_df, users_df)

C:\Users\danie\AppData\Local\Temp\ipykernel_30940\711154299.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return trips_filtered.groupby('request_at').apply(


,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


## Aditional Notes
I practiced the groupby, merge and filtering concepts in pandas by solving this excercise! 

Please note my implementation of the ```apply()``` function that allowed me to access every group created by the ```groupby()``` function and then perform the cancellation rate calculation as well! (I ended up accessing each group's first date str as my output's index, and  ```.shape[0]``` to count the number of resultant rows after applying the filter that determined whether the trip was cancelled or not).

Thank you for reading along, I loved this excercise!